---
# Running GS 
---

- Main Purpose : use the functions defined in Equations.py and run a grid search
- functions are imported -> parameter grid is defined -> code to store files is written 

- Code is in the first hald of the notebook , beyond "TESTING BEYOND THIS CELL" is just random testing 

In [2]:
import pandas as pd
import numpy
import importlib
import itertools


from Equations_Run_Combo_V_2 import *

In [3]:



cc = {
    "use_USO_wticoncat_predictor_WEEKLY_END_MO" : False , "use_UCO_wticoncat_predictor_WEEKLY_END_MO" : True ,
    "use_HUC_wticoncat_predictor_WEEKLY_END_MO" : False , "use_HOD_wticoncat_predictor_WEEKLY_END_MO" : False ,
    "use_CRUD_wticoncat_predictor_WEEKLY_END_MO" : False , "use_SCO_wticoncat_predictor_WEEKLY_END_MO" : False ,

        "learning_rate": 0.001, "num_epochs": 10,
        "batch_size": 32, "use_bidirectional": False,
        "lag": 2, "input_size": 12,
        "hidden_size": 20, "num_layers": 2,

    "use_monthly_dfs_only": True,

"use_binary_0_1_retRate": False,

        "use_binary_neg1_1": False,
"use_ret_rate": False,
        "use_print_acc": False,"use_dropout": False,
        # iter_per_valSET: int,
                                                        "use_class_weighting": True, 
        "is_deterministic": True,"seed_num": 42,

        "use_existing_lagged_data": True, "use_dynamic_weights": False    ,

        "use_binary_0_1_retRate_custom_neg": False,
        "use_binary_0_1_retRate_custom_pos": True,


                    "binary_0_1_cutoff_ret_rate_percentage": 0.1,  ### cutoff for the  use_binary_0_1_retRate_custom_pos ot use_binary_0_1_retRate_custom_neg

        "POS_weight_multiplier": 1,

        "use_rolling_fixed_train_size": False, "use_existing_initial_weights": False,"state_dict": None,

                "use_custom_loss_function_BCE_THRESH": False, # NEW NEW NE
                "use_custom_loss_function_BCE_THRESH_AND_SEVERITY": False, # NEW NEW NEW
                "use_LOW_weights_for_BCE_custom_loss": False, # NEW NEW NEW
                "pred_threshold_sigmoid01_up": None ,
                
                
            
        'train_start_month'  : "2004-01",
        'val_start_month'  : '2020-01' ,    # test_start_month = '2022-01' ; test_end_month = '2022-12' 
        'val_end_month'  : '2021-12' ,
        'num_preds_per_fold' : 3} # NEW NEW NEW



r , w = run_combo_V_4(0, cc, 4, use_print_acc_vs_pred=False , pred_threshold_sigmoid01_up_bool = False )



--- Running Combo 5 ---Parameters: {'use_USO_wticoncat_predictor_WEEKLY_END_MO': False, 'use_UCO_wticoncat_predictor_WEEKLY_END_MO': True, 'use_HUC_wticoncat_predictor_WEEKLY_END_MO': False, 'use_HOD_wticoncat_predictor_WEEKLY_END_MO': False, 'use_CRUD_wticoncat_predictor_WEEKLY_END_MO': False, 'use_SCO_wticoncat_predictor_WEEKLY_END_MO': False, 'learning_rate': 0.001, 'num_epochs': 10, 'batch_size': 32, 'use_bidirectional': False, 'lag': 2, 'input_size': 12, 'hidden_size': 20, 'num_layers': 2, 'use_monthly_dfs_only': True, 'use_binary_0_1_retRate': False, 'use_binary_neg1_1': False, 'use_ret_rate': False, 'use_print_acc': False, 'use_dropout': False, 'use_class_weighting': True, 'is_deterministic': True, 'seed_num': 42, 'use_existing_lagged_data': True, 'use_dynamic_weights': False, 'use_binary_0_1_retRate_custom_neg': False, 'use_binary_0_1_retRate_custom_pos': True, 'binary_0_1_cutoff_ret_rate_percentage': 0.1, 'POS_weight_multiplier': 1, 'use_rolling_fixed_train_size': False, 'use_

In [4]:
###                             FIRST

import math 
from joblib import parallel_backend , Parallel , delayed 
import random 


######    GS 


param_grid = {
                      "binary_0_1_cutoff_ret_rate_percentage" : [0.1],
    "learning_rate": [0.05, 0.005, 0.0005 , 0.00005],
    "num_epochs": [ 70, 150, 300, 700],
    "batch_size": [10, 30, 50, 70],
    "use_bidirectional": [False],
    "lag": [ 2, 3, 6 , 8],
    "input_size": [12],
    "hidden_size": [ 12, 20, 35, 55],
    "num_layers": [1, 2, 4 , 6],
                                            "use_monthly_dfs_only": [True],
    
                                            "use_binary_0_1_retRate": [False],

                "use_custom_loss_function_BCE_THRESH": [False , True], # NEW NEW NEW
                "use_custom_loss_function_BCE_THRESH_AND_SEVERITY": [False], # NEW NEW NEW
                "use_LOW_weights_for_BCE_custom_loss": [True], # NEW NEW NEW
                "pred_threshold_sigmoid01_up": [None], # NEW NEW NEW


    "use_binary_neg1_1": [False],  
    "use_ret_rate": [False],       
    "use_print_acc": [False],      
    "use_dropout": [False],  
    # "iter_per_valSET": [1],        
                                             "use_class_weighting": [True, False],

    "is_deterministic": [True],     "seed_num": [42],       
    "use_existing_lagged_data": [True],'use_dynamic_weights' : [False] ,


    'use_binary_0_1_retRate_custom_neg' : [False] ,
    'use_binary_0_1_retRate_custom_pos' : [True] ,
    

    'POS_weight_multiplier' : [  .1 ,  .7 ,  1  ,  1.3 , 1.5   ] , ## 24 vals is val set, 3 units per ser fi folds are 8 
    'use_rolling_fixed_train_size' : [False],
    'use_existing_initial_weights': [False], 
    'state_dict' : [None],

    "use_USO_wticoncat_predictor_WEEKLY_END_MO" : [False] , "use_UCO_wticoncat_predictor_WEEKLY_END_MO" : [False] , ### NEW NEW
    "use_HUC_wticoncat_predictor_WEEKLY_END_MO" : [False] , "use_HOD_wticoncat_predictor_WEEKLY_END_MO" : [True] , ### HOD 
    "use_CRUD_wticoncat_predictor_WEEKLY_END_MO" : [False] , "use_SCO_wticoncat_predictor_WEEKLY_END_MO" : [False] ,

                    'train_start_month'  : ["2004-01" , "2008-01"],
                    'val_start_month'  : ['2020-01'] ,    # test_start_month = '2022-01' ; test_end_month = '2022-12' 
                    'val_end_month'  : ['2021-12'] ,
                    'num_preds_per_fold' : [3] , 

  } # NEW NEW NEW

# --- Prepare all combinations ---
keys, values = zip(*param_grid.items())
combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

#### remove repeats - this happends if there is a custom loss fucntion adn a POS weight multiplier , so remove all but one 
#### of the POS weight multipliers if a custom loss function is used
### ALSO remove all with use_class_weighting = False  since there are repeats for true adn false 
#######SO jsut keep when POS_weight_multiplier = 1 if use_custom_loss_function_BCE_THRESH = True to avoid repreats 
# wehn using custom loss function

print(f"Total combinations before removing repeats: {len(combinations)}")

### run in parrallel or too time consuming

def remove_items_with_condition(combos):

    for i in combos[:]:

        if (i['use_custom_loss_function_BCE_THRESH'] == True) :
            if (i['POS_weight_multiplier'] != 1) and (i['use_class_weighting'] != True) :
                combos.remove(i)
    
    return combos


splits  = np.linspace(0, len(combinations), num=8, dtype=int).tolist()

for i in range(len(splits)-1):
    globals()[f"combos_part_{i}"] = combinations[splits[i]:splits[i+1]]

with parallel_backend("loky", n_jobs=-1):
    res = Parallel()(
        delayed(remove_items_with_condition)(globals()[f"combos_part_{i}"])
        for i in range(len(splits)-1)
    )

combos_concat_HOD = []
for i in res:
    combos_concat_HOD.extend(i)

print(f"Total combinations after removing repeats: {len(combos_concat_HOD)}")


combos_concat_UCO = []
for combo in copy.deepcopy(combos_concat_HOD):
    combo["use_UCO_wticoncat_predictor_WEEKLY_END_MO"] = True
    combo["use_HOD_wticoncat_predictor_WEEKLY_END_MO"] = False
    combos_concat_UCO.append(combo)


# combos_concat_UCO[0]

Total combinations before removing repeats: 163840
Total combinations after removing repeats: 131072


In [5]:


def random_subset(data,  seed=42):
    random.seed(seed)
    n_keep = int(len(data))
    return random.sample(data, n_keep)


# --- Split combinations into chunks ---
def split_into_chunks(lst, n_chunks):
    chunk_size = len(lst) // n_chunks
    chunks = []
    for i in range(n_chunks):
        start = i * chunk_size
        end = (i + 1) * chunk_size if i < n_chunks - 1 else len(lst)
        chunks.append(lst[start:end])
    return chunks


random_subset_combos_HOD = random_subset(combos_concat_HOD, seed=42)

random_subset_combos_UCO = random_subset(combos_concat_UCO, seed=1000)



# FRACTION_TO_KEEP = 1  ; fraction_idx = int(len(random_subset_combos) * FRACTION_TO_KEEP)


n_chunks = 20

combo_chunks_HOD = split_into_chunks(random_subset_combos_HOD[:], n_chunks)
combo_chunks_UCO = split_into_chunks(random_subset_combos_UCO[:], n_chunks)

# combo_chunks = split_into_chunks(combinations, n_chunks)

chunk_to_run_HOD = 0 #    <<------- 
chunk_to_run_UCO = 0 #    <<-------

selected_chunk_HOD = combo_chunks_HOD[chunk_to_run_HOD]
selected_chunk_UCO = combo_chunks_UCO[chunk_to_run_UCO]


In [8]:
len([combo for combo in selected_chunk_HOD if combo['use_custom_loss_function_BCE_THRESH'] == True])


2423

In [5]:

print(f"Running chunk {chunk_to_run_HOD+1} of {n_chunks} ({len(selected_chunk_HOD)} combos)")

# --- Calculate offset ---
total_offset_HOD = sum(len(chunk) for chunk in combo_chunks_HOD[:chunk_to_run_HOD])

# --- Run in parallel ---
with parallel_backend("loky", n_jobs=-1):
    results_HOD = Parallel()(
        delayed(run_combo_V_4)(i, combo, total_offset_HOD, use_print_acc_vs_pred=False , pred_threshold_sigmoid01_up_bool = False)
        for i, combo in enumerate(selected_chunk_HOD)
    )

# --- Unzip results ---
metrics_list_HOD, weights_list_HOD = zip(*results_HOD)

Running chunk 1 of 20 (6553 combos)
--- Running Combo 2 ---Parameters: {'binary_0_1_cutoff_ret_rate_percentage': 0.1, 'learning_rate': 0.05, 'num_epochs': 70, 'batch_size': 30, 'use_bidirectional': False, 'lag': 6, 'input_size': 12, 'hidden_size': 20, 'num_layers': 4, 'use_monthly_dfs_only': True, 'use_binary_0_1_retRate': False, 'use_custom_loss_function_BCE_THRESH': False, 'use_custom_loss_function_BCE_THRESH_AND_SEVERITY': False, 'use_LOW_weights_for_BCE_custom_loss': True, 'pred_threshold_sigmoid01_up': None, 'use_binary_neg1_1': False, 'use_ret_rate': False, 'use_print_acc': False, 'use_dropout': False, 'use_class_weighting': False, 'is_deterministic': True, 'seed_num': 42, 'use_existing_lagged_data': True, 'use_dynamic_weights': False, 'use_binary_0_1_retRate_custom_neg': False, 'use_binary_0_1_retRate_custom_pos': True, 'POS_weight_multiplier': 1, 'use_rolling_fixed_train_size': False, 'use_existing_initial_weights': False, 'state_dict': None, 'use_USO_wticoncat_predictor_WEEKLY

KeyboardInterrupt: 

In [6]:

print(f"Running chunk {chunk_to_run_UCO+1} of {n_chunks} ({len(selected_chunk_UCO)} combos)")

# --- Calculate offset ---
total_offset_UCO = sum(len(chunk) for chunk in combo_chunks_UCO[:chunk_to_run_UCO])

# --- Run in parallel ---
with parallel_backend("loky", n_jobs=-1):
    results_UCO = Parallel()(
        delayed(run_combo_V_4)(i, combo, total_offset_UCO, use_print_acc_vs_pred=False , pred_threshold_sigmoid01_up_bool = False)
        for i, combo in enumerate(selected_chunk_UCO)
    )

# --- Unzip results ---
metrics_list_UCO, weights_list_UCO = zip(*results_UCO)

Running chunk 1 of 20 (6553 combos)
--- Running Combo 5 ---Parameters: {'binary_0_1_cutoff_ret_rate_percentage': 0.1, 'learning_rate': 0.005, 'num_epochs': 300, 'batch_size': 30, 'use_bidirectional': False, 'lag': 2, 'input_size': 12, 'hidden_size': 55, 'num_layers': 1, 'use_monthly_dfs_only': True, 'use_binary_0_1_retRate': False, 'use_custom_loss_function_BCE_THRESH': False, 'use_custom_loss_function_BCE_THRESH_AND_SEVERITY': False, 'use_LOW_weights_for_BCE_custom_loss': True, 'pred_threshold_sigmoid01_up': None, 'use_binary_neg1_1': False, 'use_ret_rate': False, 'use_print_acc': False, 'use_dropout': False, 'use_class_weighting': True, 'is_deterministic': True, 'seed_num': 42, 'use_existing_lagged_data': True, 'use_dynamic_weights': False, 'use_binary_0_1_retRate_custom_neg': False, 'use_binary_0_1_retRate_custom_pos': True, 'POS_weight_multiplier': 1.3, 'use_rolling_fixed_train_size': False, 'use_existing_initial_weights': False, 'state_dict': None, 'use_USO_wticoncat_predictor_WEE

KeyboardInterrupt: 

In [ ]:
import json

file_json_metrics_HOD = f"GC_full_results_chunk_{chunk_to_run_HOD + 1}_{n_chunks}_CHUNKS_8fold_10percentPOS_V_1_HOD.json"
# file_json_metrics = f"/Users/cs/Desktop/full_results_chunk_{chunk_to_run + 1}_{n_chunks}_CHUNKS_8fold_05NEG__VERSION_2.json"

file_weights_HOD = f"GC_full_results_chunk_{chunk_to_run_HOD + 1}_{n_chunks}_WEIGHTS_8fold_10percentPOS_V_1_HOD.pt"
# file_weights = f"/Users/cs/Desktop/full_results_chunk_{chunk_to_run + 1}_{n_chunks}_WEIGHTS_8fold_05NEG__VERSION_2.pt"


with open(file_json_metrics_HOD, "w") as f_json:
    json.dump(metrics_list_HOD, f_json, indent=2)



# weights_list = copy.deepcopy(weights_list)

# torch.save(weights_list, file_weights)        



In [ ]:
import json

file_json_metrics_UCO = f"GC_full_results_chunk_{chunk_to_run_UCO + 1}_{n_chunks}_CHUNKS_8fold_10percentPOS_V_1_UCO.json"
# file_json_metrics = f"/Users/cs/Desktop/full_results_chunk_{chunk_to_run + 1}_{n_chunks}_CHUNKS_8fold_05NEG__VERSION_2.json"

file_weights_UCO = f"GC_full_results_chunk_{chunk_to_run_UCO + 1}_{n_chunks}_WEIGHTS_8fold_10percentPOS_V_1_UCO.pt"
# file_weights = f"/Users/cs/Desktop/full_results_chunk_{chunk_to_run + 1}_{n_chunks}_WEIGHTS_8fold_05NEG__VERSION_2.pt"


with open(file_json_metrics_UCO, "w") as f_json:
    json.dump(metrics_list_UCO, f_json, indent=2)




---
# TESTING BEYOND THIS CELL
---